# <center>Workflow for on-line GC and HPLC analysis in flow chemistry</center>


---
## Section 0: Imports, Paths, and Logging
---

In this section all the necessary python packages are imported, the path to this notebook and the logger for this notebook is set up.

In [5]:
from sklearn.linear_model import LinearRegression

a = LinearRegression()


In [9]:
a.__name__

AttributeError: 'LinearRegression' object has no attribute '__name__'

In [1]:
# Activate autoreload to keep on track with changing modules #
%reload_ext autoreload
%autoreload 2

# Import standard libraries #
import numpy as np
import logging
import json
from pathlib import Path
from sdRDM import DataModel

# Import tools for data processing and analysis #
from datamodel.tools import initialize_dataset
from datamodel.tools import reading_raw_data_widget
from datamodel.tools import analyzing_raw_data_widget


In [2]:
from datamodel.core import Experiment
from datamodel.tools.readers.gcparser import gc_parser
from datamodel.tools.readers.gstaticparser import gstatic_parser
from datamodel.tools.readers.mfmparser import mfm_parser

from datamodel.tools.calculus.faraday_efficiency_calculator import FaradayEfficiencyCalculator

In [3]:
# Define paths for loggin output #
root                = Path.cwd()
logging_config_path = root / "datamodel_b07_tc/tools/logging/config.json"

# Read in logger specs and configurate logger (set name to current notebook) #
with open(logging_config_path) as logging_config_json: logging.config.dictConfig( json.load( logging_config_json ) )
logger = logging.getLogger(__name__)

# Set the level of thid-party logger to avoid dumping too much information #
for logger_ in ['markdown_it', 'h5py', 'numexpr', 'git']: logging.getLogger(logger_).setLevel('WARNING')

In [21]:
e_chem = str(root)+'/data/Rohdaten/01_EChem/CAD14-Cu@AB/GSTATIC.DTA'
mfm    = str(root)+'/data/Rohdaten/03_MFM/CAD14-Cu@AB/Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv'
gc     = [str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/REPORT01.CSV']
calib  = str(root)+'/data/calibration/calibration.json'
correc = str(root)+'/data/correction_factors/correction_factors.json'
farada = str(root)+'/data/faraday_coefficients/faraday_coefficients.json'

experiment                 = Experiment()
        
potentiostatic_measurement = gstatic_parser( metadata_path = e_chem )
mfm_measurement            = mfm_parser( experimental_data_path = mfm )
gc_measurements_list       = [ gc_parser( metadata_path = gc[i], experimental_data_path = gc[i+1] ) for i in range( 0, len(gc), 2 ) ]

experiment.measurements    = [ potentiostatic_measurement, mfm_measurement, *gc_measurements_list ]

# Read in parameters such as calibration, correction factors and farraday coefficients and save it in Experiment class #
experiment.calibrate_from_json( calib, degree=1 )
experiment.read_correction_factors( correc )
experiment.read_faraday_coefficients( farada )


ass_dict = {"Hydrogen":[69.1716],"Carbon dioxide":[65492.7461],"Carbon monoxide":[2876.9526,43.7317],
            "Methane":[164.157,141.1739],"Ethene":[1624.0737],"Ethane":[]}


fe = FaradayEfficiencyCalculator(experiment=experiment)
fe.calculate_faraday_efficiencies( experiment.measurements[2].get("metadata","parameter","Injection Date")[0][0].value, ass_dict )

---
## Section 1: Dataset and raw data
---
In this section the dataset as well as the to analyze raw data is choosen

In [ ]:
# To do:
# 0) If dataset is saved and read in again, the dataset has the object "equipment", what is not there in the modified verison
#    this will cause an error
# 1) DaRUS extension


In [4]:
git_path = 'https://github.com/FAIRChemistry/datamodel_b07_tc.git.samir_develop'

id = initialize_dataset()
id.write_dataset(root,git_path)

ValueError: Could not find object 'LinearRegression' in objects.

In [4]:
## Definition of basic meta data of the project ##

id.title.value        = 'Electrocatalytic CO2-reduction on carbon'
id.description.value  = 'The aim of this project is to blablabla'

# List with authors and their affiliation #
id.authors.value      = 'Richard Schömig, Maximilian Schmidt' 
id.affiliations.value = 'University of Stuttgart, University of Stuttgart'
id.dataset_text.value = "test1"

In [6]:
## Search for dataset and raw data ##

rrdw = reading_raw_data_widget()
rrdw.choose_data(root)

KeyError: 'type'

In [4]:
#str(root) = "c:/Users/darouich/OneDrive/Dokumente/"

e_chem = str(root)+'/data/Rohdaten/01_EChem/CAD14-Cu@AB/GSTATIC.DTA'
mfm    = str(root)+'/data/Rohdaten/03_MFM/CAD14-Cu@AB/Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv'
gc     = [str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/REPORT01.CSV']
calib  = str(root)+'/data/calibration/calibration.json'
correc = str(root)+'/data/correction_factors/correction_factors.json'
farada = str(root)+'/data/faraday_coefficients/faraday_coefficients.json'

rrdw.Echem_files.value = [e_chem]
rrdw.MFM_files.value   = [mfm]
rrdw.GC_files.value    = gc
rrdw.calib_files.value = [calib]
rrdw.correction_files.value = [correc]
rrdw.faraday_files.value    = [farada]

NameError: name 'rrdw' is not defined

---
## Section 2: Analysis of data
---
In this section the raw data of the above choosen dataset is analyzed

In [14]:
ardw = analyzing_raw_data_widget()
ardw.choose_experiment(rrdw.datamodel,rrdw.dataset_dropdown.value)


Starting the postprocessing

Faraday effiencies of GC measurement n°0
                Faraday_efficiency
Carbon dioxide          256.959414
Carbon monoxide           0.382582
Ethane                    0.436018
Ethene                   12.724777
Hydrogen                 15.177137
Methane                   6.567519 

Faraday effiencies of GC measurement n°1
                Faraday_efficiency
Carbon dioxide          264.135754
Carbon monoxide           0.669252
Ethane                     0.43215
Ethene                   11.360659
Hydrogen                   20.3274
Methane                   6.419533 

Faraday effiencies of GC measurement n°2
                Faraday_efficiency
Carbon dioxide          275.513076
Carbon monoxide           0.690107
Ethane                    0.381448
Ethene                    9.899592
Hydrogen                 19.704288
Methane                   6.618435 


Mean Faraday efficency over all GC measurements
                Faraday_efficiency
Carbon dioxide        

---
## Section 3: Upload of data to DaRUS
---
In this section the processed, as well as the raw data is uploaded to DaRUS

In [75]:
from pyDaRUS import Citation
from pyDaRUS import Dataset
from pyDaRUS.metadatablocks.citation import SubjectEnum, IdentifierScheme

subject_list = [ subject.value for subject in SubjectEnum]

# Initialize the metadatablock
citation = Citation()

citation.title = "My Title"

citation.add_description(text="test")
citation.add_author(name="Samir", affiliation="ITT")
citation.add_contact(name="Samir", email="samir.darouich@itt.uni-stuttgart.de")

citation.subject = ["Engineering"]

citation.add_grant_information( grant_agency="DFG", grant_number="358283783 - SFB 1333")
citation.add_project( name="test project", level=1 )
citation.depositor = "XXX"
citation.deposit_date = "XXX"


dataset = Dataset()
dataset.add_metadatablock(citation)

dataset.add_file(dv_path=".", local_path="my.file")

#https://darus.uni-stuttgart.de/api/dataverses/sfb1333

dataset.upload(dataverse_name="sfb1333-hansen-gross",
               DATAVERSE_URL="https://darus.uni-stuttgart.de",
               API_TOKEN="4afecd82-c92d-4935-b786-2225af43531e")

Attribute project not valid for import (dv_up).


Exception: Validation Failed: Point of Contact Name is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Description Text is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Project Level is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Funding Information Identifier is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Project Name is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Deposit Date is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Author Name is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Depositor is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Point of Contact E-mail is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]), Funding Information Agency is required. (Invalid value:edu.harvard.iq.dataverse.DatasetField[ id=null ]